# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import iiif_prezi3
import pandas as pd
import json
import os
from tqdm import tqdm
import os

In [ ]:
#| export

class TojiClient:

    def __init__(self, data_dir, docs_dir, origin, images, image_metadata_map, document_metadata_map, image_size, is_debug=False, is_skip_if_manifest_exists=True):

        self.data_dir = data_dir
        self.docs_dir = docs_dir

        self.origin = origin

        self.images = images
        self.image_metadata_map = image_metadata_map
        self.document_metadata_map = document_metadata_map
        self.image_size = image_size

        self.is_debug = is_debug
        self.is_skip_if_manifest_exists = is_skip_if_manifest_exists

        self.attribution = "京都府立京都学・歴彩館 東寺百合文書WEB"
        self.rights = "https://creativecommons.org/licenses/by/2.1/jp/"
        self.homepage = "https://hyakugo.pref.kyoto.lg.jp/"

    def createManifest(self, row_d):

        origin = self.origin

        d_id = row_d['文書ID']        

        opath = f"{self.docs_dir}/iiif/3/{d_id}/manifest.json"

        if self.is_skip_if_manifest_exists and os.path.exists(opath):
            with open(opath, "r") as f:
                try:
                    manifest_json = json.load(f)

                    manifest = iiif_prezi3.Manifest(
                        id=manifest_json['id'],
                        label=manifest_json['label'],
                    )

                    return manifest
                except Exception as e:
                    print(opath, e)
        
        prefix = f"{origin}/3/{d_id}"
        
        document_metadata_map = self.document_metadata_map
        image_metadata_map = self.image_metadata_map

        os.makedirs(os.path.dirname(opath), exist_ok=True)

        metadata = []

        for c, col in document_metadata_map[d_id].items():
            
            if c == "文書ID":
                continue
            if pd.notna(col):
                metadata.append(iiif_prezi3.KeyValueString(label=c, value=col))

        
        for c, col in image_metadata_map[d_id].items():
            if c in ["和暦", "西暦年", "月", "日", "東寺文書検索システム番号"]:
                if pd.notna(col):
                    metadata.append(iiif_prezi3.KeyValueString(label=c, value=col))
        
        title = f'{row_d["文書コード"]}：{row_d["タイトル"]}'

        manifest = iiif_prezi3.Manifest(
            id=f"{prefix}/manifest.json", 
            label=title, 
            metadata=metadata,
            rights=self.rights,
            requiredStatement=iiif_prezi3.KeyValueString(label="Attribution", value=self.attribution),
            viewingDirection="right-to-left",
            provider=iiif_prezi3.ProviderItem(
                id=self.homepage,
                label=self.attribution,
            ),
            homepage=iiif_prezi3.HomepageItem(
                id=f"{self.homepage}contents/detail.php?id={d_id}",
                type="Text",
                label=f"{title}｜文書詳細｜東寺百合文書",
                format="text/html",
                language="ja"
            ),
        )
        
        manifest = self.createCanvases(manifest, row_d, prefix)

        return manifest

    def createCanvases(self, manifest, row_d, prefix):

        image_metadata_map = self.image_metadata_map
        image_size = self.image_size
        images = self.images

        d_id = row_d["文書ID"]

        item_id = image_metadata_map[d_id]["アイテムID"]

        for i, image in images[images["アイテムID"] == item_id].iterrows():

            image_url = image["image_l"]

            matching_image = next((img for img in image_size if img['url'] == image_url), None)

            canvas_width = matching_image["width"]
            canvas_height = matching_image["height"]

            image_id = f"{item_id}-{image['file_index']}"

            canvas = manifest.make_canvas(
                id=f"{prefix}/canvas/{image_id}",
                label=f"[{image_id}]",
            )

            anno_body = iiif_prezi3.ResourceItem(
                id=image_url,
                type="Image",
                format="image/jpeg"
            )

            anno_page = iiif_prezi3.AnnotationPage(
                id=f"{prefix}/annotation-page/{image_id}"
            )

            anno = iiif_prezi3.Annotation(
                id=f"{prefix}/annotation/{image_id}",
                motivation="painting",
                body=anno_body,
                target=canvas.id
            )

            anno_body.set_hwd(height=canvas_height, width=canvas_width)

            canvas.set_hwd(height=canvas_height, width=canvas_width)

            canvas.thumbnail = iiif_prezi3.ResourceItem(
                id=image["image_s"],
                type="Image",
                format="image/jpeg",
            )

            anno_page.add_item(anno)

            canvas.add_item(anno_page)

        opath = f"{self.docs_dir}/iiif/3/{d_id}/manifest.json"

        with open(opath, "w") as f:
            f.write(manifest.json(ensure_ascii=False, indent=2 if self.is_debug else None))

        return manifest
    
    def createCollection(self, rows): # , rows

        origin = self.origin
        IS_DEBUG = self.is_debug

        collection = iiif_prezi3.Collection(
            id=f"{origin}/set/3/collection.json",
            label="東寺百合文書",
            viewingDirection="right-to-left",
            provider=iiif_prezi3.ProviderItem(
                id=self.homepage,
                label=self.attribution,
            ),
            homepage=iiif_prezi3.HomepageItem(
                id=self.homepage,
                type="Text",
                label=self.attribution,
                format="text/html",
                language="ja"
            ),
            metadata=[
                iiif_prezi3.KeyValueString(label="Attribution", value=self.attribution),
                iiif_prezi3.KeyValueString(label="Rights", value=self.rights),
            ],
            rights=self.rights,
        )

        for row_d in tqdm(rows):

            manifest = self.createManifest(row_d)

            collection.items.append({
                "id": manifest.id,
                "label": manifest.label,
                "type": "Manifest",
            })

        opath = f"{self.docs_dir}/iiif/set/3/collection.json"

        os.makedirs(os.path.dirname(opath), exist_ok=True)

        with open(opath, "w") as f:
            f.write(collection.json(ensure_ascii=False, indent=2 if IS_DEBUG else None))

    @staticmethod
    def main(data_dir, docs_dir, origin, size=-1, is_debug=False, is_skip_if_manifest_exists=True):

        data_dir = os.path.normpath(data_dir)
        docs_dir = os.path.normpath(docs_dir)

        iiif_prezi3.config.configs['helpers.auto_fields.AutoLang'].auto_lang = "ja"

        images = pd.read_csv(f"{data_dir}/tjhm_docs_20200130/images.txt", sep="\t").fillna("").astype(str)

        image_metadata = pd.read_csv(f"{data_dir}/tjhm_docs_20200130/documents1.txt", sep="\t")

        image_metadata['東寺文書検索システム番号'] = image_metadata['東寺文書検索システム番号'].fillna(0).astype(int).astype(str)

        image_metadata = image_metadata.fillna("").astype(str)

        document_metadata = pd.read_csv(f"{data_dir}/tjhm_docs_20200130/documents2.txt", sep="\t").fillna("").astype(str)

        image_metadata_map = {}

        for i, row in image_metadata.iterrows():
            
            id = row["文書ID"]

            image_metadata_map[id] = row

        document_metadata_map = {}

        for i, row in document_metadata.iterrows():
            
            id = row["文書ID"]

            document_metadata_map[id] = row

        with open(f"{data_dir}/image_sizes.json", "r") as f:
            image_size = json.load(f)
        
        rows = []

        for d, row_d in document_metadata.iterrows():

            if size > 0 and len(rows) >= size:
                break

            rows.append(row_d)

        client = TojiClient(data_dir, docs_dir, origin, images, image_metadata_map, document_metadata_map, image_size, is_debug=is_debug, is_skip_if_manifest_exists=is_skip_if_manifest_exists)

        client.createCollection(rows)

    @staticmethod
    def create_index(path, opath):
        with open(path) as f:
            data = json.load(f)

        # HTMLの開始
        html = "<html><body>"

        html += f"<h1>{data['label']['ja'][0]}</h1>"

        html += "<table>"

        html += "<tbody>"

        # 各人物のデータを行として追加
        for item in data["items"]:
            html += f"""<tr>
            <td>
            <a href="./mirador/?manifest={item['id']}" target="_blank">
            {item['label']['ja'][0]}
            </a>
            </td>
            </tr>"""

        # HTMLの終了

        html += "<tbody>"

        html += "</table>"
        html += "</body></html>"

        # HTMLをファイルに保存
        with open(opath, 'w') as file:
            file.write(html)


In [ ]:
show_doc(TojiClient.main)

---

[source](https://github.com/nakamura196/toji_iiif/blob/main/toji_iiif/core.py#L220){target="_blank" style="float:right; font-size:smaller"}

### TojiClient.main

>      TojiClient.main (data_dir, docs_dir, origin, size=-1, is_debug=False,
>                       is_skip_if_manifest_exists=True)

In [ ]:
show_doc(TojiClient.createCollection)

---

[source](https://github.com/nakamura196/toji_iiif/blob/main/toji_iiif/core.py#L175){target="_blank" style="float:right; font-size:smaller"}

### TojiClient.createCollection

>      TojiClient.createCollection (rows)

|    | **Details** |
| -- | ----------- |
| rows | , rows |

In [ ]:
show_doc(TojiClient.createManifest)

---

[source](https://github.com/nakamura196/toji_iiif/blob/main/toji_iiif/core.py#L37){target="_blank" style="float:right; font-size:smaller"}

### TojiClient.createManifest

>      TojiClient.createManifest (row_d)

In [ ]:
show_doc(TojiClient.createCanvases)

---

[source](https://github.com/nakamura196/toji_iiif/blob/main/toji_iiif/core.py#L107){target="_blank" style="float:right; font-size:smaller"}

### TojiClient.createCanvases

>      TojiClient.createCanvases (manifest, row_d, prefix)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()